# Using Forms

Now let's see how we can use HTML forms to submit our own data.

HTML forms allow us to get user entries and then use the submitted values as parameters in our code. (See [W3 Schools](https://www.w3schools.com/html/html_forms.asp) for more details.)

Suppose that you want to have the ability to search the Citibike stations by their name, and get back a list of the stations that contain the search string. A simple HTML form that can do that is listed below. Save the file  `search_stations.html` in your `templates` folder, and add a new route in your webserver.py, so that you can get back this page, when you visit the main page of your website.


#### `search_stations.html`

```html
<!DOCTYPE html>
<html>
<head>
    <title>Search Citibike Stations</title>
</head>
<body>
      <form action="/search">
      Station Name:<br>
      <input type="text" name="name"><br>
    </form>
</body>
```

#### `webserver.py`

```python
@app.route('/')
def home():
    return render_template('search_stations.html')
```

A key part of the form is the `action` script.

This defines what is the URL that we should go to, when we submit the form. In our case, it will go to the `http://<your IP>:5000/search` with the parameter `name` having the value of the text that the user entered.

For example, if we enter `Bleecker` on the form, the form will direct us to the URL `http://<your IP>:5000/search?name=Bleecker`.

Now, we need to specify the corresponding route in our webserver (i.e., the `/search` route) and define what it will do.

Below we list the code that needs to be added in the `webserver.py` to implement that route.

#### `webserver.py`

```python
@app.route('/search')
def search():

    name = request.args.get('name')

    conn_string = 'mysql://{user}:{password}@{host}/{db}?charset={encoding}'.format(
        host = 'db.ipeirotis.org', 
        user = 'student',
        db = 'citibike_fall2017',
        password = 'dwdstudent2015',
        encoding = 'utf8mb4')
    engine = create_engine(conn_string)

    con = engine.connect()
    query = '''SELECT DISTINCT id AS station_id, name, capacity, lat, lon
               FROM status_fall2017
               WHERE name LIKE %s
            '''
    stations = con.execute(query, ('%'+name+'%',))
    con.close()
    
    return render_template('citibike.html', stations=stations)
```